This script do:
- preprocess:
    - reset time zone to Asia/Shanghei 
    -
- add features
   - mean encode 
       - by (ip,app,device,os)
    
- features base
    - use only (gain importance)
        - app_mean_target
        - app
        - channel_mean_target
        - next_click_dt
        - channel_nunique_ip
        - os
        - channel
        - hour
        - app_nunique_channel
        - channel_cnt_ip_app
        - hour_nunique_day_ip
        - app_nunique_ip
        - app_nunique_ip_device_os
        - device_mean_target 	
        - channel_cnt_ip_day_hour
        - app_cumcnt_ip_device_os
        - ip_mean_target
        - os_mean_target

In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import pytz
%matplotlib inline 
import gc,time

In [3]:
with pd.HDFStore('../input/raw_data.h5') as store:
    print(store.keys())
    test_df = store['test_df']
    train_df = store['train_df']

['/test_df', '/train_df']


____

In [4]:
mask = (train_df.day>7) & (train_df.day<10)
trn_df = train_df[mask]

In [5]:
len_trn = len(trn_df)
trn_df = trn_df.append(test_df)

del train_df;gc.collect()

7

In [6]:
print('trn_df :{}'.format(trn_df.shape)) # 
# print('test_df:{}'.format(test_df.shape))
del test_df

trn_df :(143984060, 10)


- len_trn  : 125193591 

- len_tst  : 18790469

# helper func

In [7]:
def encode_agg_feature(train_df, selcols, groupby, aggregator = 'nunique'):
    usecols = [e for e in selcols if e not in  groupby]    
    
    if aggregator == 'nunique':  
        colname= usecols[-1] + '_nunique_' + '_'.join(groupby)
        if colname in train_df.columns:
            train_df.drop(colname,axis=1,inplace=True)
            
        gp = train_df[selcols].groupby(groupby)[usecols].nunique().reset_index().\
            rename(columns = {
                usecols[-1] : usecols[-1] + '_nunique_' + '_'.join(groupby)
            })
        train_df = train_df.merge(gp, how='left', on=groupby)

    elif aggregator == 'cumcount':
        colname = usecols[-1] + '_cumcnt_' + '_'.join(groupby)
        if colname in train_df.columns:
            train_df.drop(colname,axis=1,inplace=True)
            
        gp = train_df[selcols].groupby(groupby)[usecols].cumcount()        
        train_df[colname] = gp.values
    
    elif aggregator == 'count':
        colname = usecols[-1] + '_cnt_' + '_'.join(groupby)
        if colname in train_df.columns:
            train_df.drop(colname,axis=1,inplace=True)
        gp = train_df[selcols].groupby(groupby)[usecols].count().reset_index().\
            rename(columns = {
                usecols[-1] : colname
            })
        train_df = train_df.merge(gp, how='left', on=groupby)
        
    elif aggregator in ['var','mean']:
        agg=np.var if aggregator == 'var' else np.mean
        
        colname = usecols[-1] + '_'+ str(aggregator) +'_' + '_'.join(groupby)
        
        if colname in train_df.columns:
            train_df.drop(colname,axis=1, inplace=True)
            
        gp = train_df[selcols].groupby(groupby).agg(agg).reset_index().\
            rename(columns = {
                usecols[-1] : colname
            })
        train_df = train_df.merge(gp, how='left', on=groupby)
    
    return train_df.iloc[:,-1]

#### feat_base             
        - [v] channel_nunique_ip (X0)
        - [v] app_nunique_channel (X6)
        - [v] channel_cnt_ip_app (X10)
        - [v] hour_nunique_day_ip (X2)
        - [v] app_nunique_ip (X3)
        - [v] app_nunique_ip_device_os (X8)
        - [v] channel_cnt_ip_day_hour (ip_tcount)
        - [v] app_cumcnt_ip_device_os (X1)
        

In [10]:
print('X0 : number of unique "channel" by ip',end='\t')
X0 = encode_agg_feature(trn_df,groupby=['ip'], selcols=['ip','channel'],aggregator='nunique')
print(X0.name)
trn_df[X0.name] = X0.values.astype('uint8') ; del X0;gc.collect()


print('X1 : cumcount of app by (ip, device, os)',end='\t')
X1 = encode_agg_feature(trn_df, selcols=['ip','device','os','app'], groupby=['ip','device','os'], aggregator='cumcount')
print(X1.name)
trn_df[X1.name] = X1.values.astype('uint16'); del X1;gc.collect()


print('X2 : number of unique "hour" by (ip, day)',end='\t')
X2 = encode_agg_feature(trn_df,selcols=['hour','ip','day'], groupby=['ip','day'], aggregator='nunique')
print(X2.name)
trn_df[X2.name] = X2.values.astype('uint8'); del X2;gc.collect()


print('X3 : number of unique "app" by ip', end='\t')
X3 = encode_agg_feature(trn_df,selcols=['app', 'ip'], groupby=['ip'])
print(X3.name)
trn_df[X3.name] = X3.values.astype('uint8'); del X3; gc.collect()



print('X6 : number of unique "app" by channel',end='\t')
X6 = encode_agg_feature(trn_df,selcols=['app', 'channel'], groupby=['channel'])
print(X6.name)
trn_df[X6.name] = X6.values.astype('uint8'); del X6; gc.collect()


print('X8 : number of unique "app" by (ip,device,os)',end='\t')
X8 = encode_agg_feature(trn_df,selcols=['app', 'ip','device','os'], groupby=['ip','device','os'])
print(X8.name)
trn_df[X8.name] = X8.values.astype('uint8'); del X8;gc.collect()

### count ####
print('X10: count by (ip,app)', end='\t')
X10 = encode_agg_feature(trn_df,selcols=['ip','app','channel'],groupby=['ip','app'],aggregator='count')
print(X10.name)
trn_df[X10.name]= X10.values.astype('uint16'); del X10;gc.collect()


print('ip_tcount',end='\t')
ip_tcount = encode_agg_feature(trn_df,selcols=['ip','day','hour','channel'], groupby=['ip','day','hour'],aggregator='count')
print(ip_tcount.name)
trn_df[ip_tcount.name]= ip_tcount.values.astype('uint16'); del ip_tcount;gc.collect()


print('complete')

X0 : number of unique "channel" by ip	channel_nunique_ip
X1: cumcount of app by (ip, device, os)	app_cumcnt_ip_device_os
X2 : number of unique "hour" by (ip, day)	hour_nunique_ip_day
X3 : number of unique "app" by ip	app_nunique_ip
X6 : number of unique "app" by channel	app_nunique_channel
X8 : number of unique "app" by (ip,device,os)	app_nunique_ip_device_os
X10 : count by (ip,app)	channel_cnt_ip_app
ip_tcount	channel_cnt_ip_day_hour
complete


importanta feature 

#### next click

- next click time by ('ip','app','os','device')

In [11]:
print('next click time by(ip,app,os,device)...')
gp = trn_df.groupby(['ip','app','os','device'])
print('shift 1...')

trn_df['shift_1'] = gp.click_time.shift(-1)
cst = pytz.timezone('Asia/Shanghai')
trn_df['shift_1'] = pd.to_datetime(trn_df['shift_1']).dt.tz_localize(pytz.utc).dt.tz_convert(cst) 
trn_df['next_click_dt'] = ((trn_df.shift_1 - trn_df.click_time) / np.timedelta64(1,'s')).fillna(50000).astype('uint16')
print('next_click_dt added')

print('gc...')
trn_df.drop(['shift_1'],axis=1,inplace=True)
gc.collect()
print('complete')

next click time by(ip,app,os,device)...
shift 1...
next_click_dt added
gc...
complete


save temp

In [12]:
store = pd.HDFStore('../input/feat/feat3_partial3.h5')
store['trn_df'] = trn_df
store.close()
    

load 

In [1]:
with pd.HDFStore('../input/feat/feat4_trn_day8_val_day9.h5') as store:
    trn_df = store['trn_df']

NameError: name 'pd' is not defined

In [3]:
len_tst = 18790469
len_trn = 143984060 - 18790469

#### mean encode ON TRAINNING ONLY NOT ON TEST ###
- app_mean_target
- channel_mean_target
- device_mean_target 	
- ip_mean_target

- (ip,app,device,os) mean target

In [4]:
# test_df = trn_df.iloc[len_trn:,]
# trn_df  = trn_df.iloc[:len_trn,]

In [5]:
debug = 0
if debug:
    len_tst = 10000
    len_trn = 10000
else:
    len_tst = 18790469
    len_trn = 143984060 - 18790469
test_df = trn_df.iloc[len_trn:,]
trn_df = trn_df.iloc[:len_trn,]

In [7]:
print('trn_df shape :',trn_df.shape)
print('test_df shape:',test_df.shape)

trn_df shape : (125193591, 19)
test_df shape: (18790469, 19)


In [8]:
from sklearn.model_selection import StratifiedKFold
print('copy trn_df to make new features')
train_new = trn_df.copy()

print('create new cols')
cols = [['ip','app','os','device'],'ip','app','channel','device','os']
for col in cols :
    if isinstance(col,list):
        col_str = '_'.join(col)
    else:
        col_str = col
    train_new[col_str + '_mean_target'] = 0

print('StratifiedKFold...')
y_tr = trn_df.is_attributed.values.astype(np.int8) # target 
skf = StratifiedKFold(4, random_state=0)
skf.get_n_splits(X=trn_df,y=y_tr)


for fold, (tr_idx, val_idx) in enumerate(skf.split(train_new,y_tr)):
    
    ## generate features 
    
    X_tr ,X_val = trn_df.iloc[tr_idx], trn_df.iloc[val_idx]
    
    print('fold:{} mean encoding...'.format(fold),end='\n\t')
#     print('tr_idx:{}'.format(tr_idx))
    for col in cols:        
        print(col,end='\t')
        if isinstance(col,str):
            means = X_val[col].map(X_tr.groupby(col).is_attributed.mean()) ## map mean encoding in X_tr to X_val
            col_str = col
            
        elif isinstance(col,list):
            col_str = '_'.join(col)
            gp = X_tr.groupby(col).is_attributed.mean().reset_index().rename(columns={'is_attributed':col_str+'_mean_target'})
            means = X_val[col].merge(gp,how='left',on=col)[col_str+'_mean_target']
            
        X_val[col_str + '_mean_target'] = means.astype('float16')
        del means;gc.collect()
        
        
        
    train_new.iloc[val_idx] = X_val
    print('')    
    del X_tr,X_val;gc.collect()
            
prior = trn_df.is_attributed.mean()

col_used = [ '_'.join(col) + '_mean_target' if isinstance(col,list) else col for col in cols]
train_new[col_used] = train_new[col_used].fillna(prior)

trn_df = train_new
del train_new;gc.collect()
print('complete')

copy trn_df to make new features
create new cols
fold:0 mean encoding...
	['ip', 'app', 'os', 'device']	

C:\Program Files\Anaconda3\envs\py36\lib\site-packages\ipykernel\__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


ip	app	channel	device	
fold:1 mean encoding...
	['ip', 'app', 'os', 'device']	ip	app	channel	device	
fold:2 mean encoding...
	['ip', 'app', 'os', 'device']	ip	app	channel	device	
fold:3 mean encoding...
	['ip', 'app', 'os', 'device']	ip	app	channel	device	
complete


In [9]:
trn_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 125193591 entries, 59709852 to 184903442
Data columns (total 24 columns):
app                             uint16
channel                         uint16
click_id                        float64
click_time                      datetime64[ns, Asia/Shanghai]
day                             uint8
device                          uint16
hour                            uint8
ip                              uint32
is_attributed                   float64
os                              uint16
channel_nunique_ip              uint8
app_cumcnt_ip_device_os         uint16
hour_nunique_ip_day             uint8
app_nunique_ip                  uint8
app_nunique_channel             uint8
app_nunique_ip_device_os        uint8
channel_cnt_ip_app              uint16
channel_cnt_ip_day_hour         uint16
next_click_dt                   uint16
ip_app_os_device_mean_target    float16
ip_mean_target                  float16
app_mean_target                 float

___ 

#### mean encode on TEST

In [29]:
cols = [['ip','app','os','device'],'ip','app','device','channel']

selcols = ['ip','app','os','device','channel','is_attributed']

for col in cols:
    print('target_mean col:{}...'.format(col))
    gp = trn_df[selcols].groupby(col)
    if isinstance(col,list):
        col_str = '_'.join(col)
        gp = gp.is_attributed.mean().reset_index().rename(columns={'is_attributed':col_str+'_mean_target'})
        means = test_df.merge(gp,how='left',on=col)[col_str+'_mean_target'].astype('float16')
    else:
        col_str = col
        means = test_df[col].map(gp.is_attributed.mean()).astype('float16')
        
    test_df[col_str +'_mean_target'] = means
    del means,gp;gc.collect()
    
prior = trn_df.is_attributed.mean()


col_used = [ '_'.join(col) + '_mean_target' if isinstance(col,list) else col for col in cols]
test_df[col_used].fillna(prior)

gc.collect()
print('complete')

target_mean col:['ip', 'app', 'os', 'device']...
target_mean col:ip...
target_mean col:app...
target_mean col:device...
target_mean col:channel...
complete


In [30]:
test_df.shape

(18790469, 24)

In [31]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18790469 entries, 0 to 18790468
Data columns (total 24 columns):
app                             uint16
channel                         uint16
click_id                        float64
click_time                      datetime64[ns, Asia/Shanghai]
day                             uint8
device                          uint16
hour                            uint8
ip                              uint32
is_attributed                   float64
os                              uint16
channel_nunique_ip              uint8
app_cumcnt_ip_device_os         uint16
hour_nunique_ip_day             uint8
app_nunique_ip                  uint8
app_nunique_channel             uint8
app_nunique_ip_device_os        uint8
channel_cnt_ip_app              uint16
channel_cnt_ip_day_hour         uint16
next_click_dt                   uint16
ip_app_os_device_mean_target    float16
ip_mean_target                  float16
app_mean_target                 float16
device

___

# Save 



In [33]:
## day8 train_sets
train_df = trn_df[trn_df.day==8]
## day9 valid sets 
val_df = trn_df[trn_df.day==9]


In [34]:
print("train shape: ", train_df.shape)
print("valid shape: ", val_df.shape)
print("test shape : ", test_df.shape)


train shape:  (62360949, 24)
valid shape:  (62832642, 24)
test shape :  (18790469, 24)


In [35]:
trn_day = 'day8'
val_day = 'day9'
print('../../feat4_trn_{}_val_{}'.format(trn_day,val_day))
store = pd.HDFStore('../input/feat/feat4_trn_{}_val_{}.h5'.format(trn_day,val_day))
store['train_df'] = train_df
store['valid_df'] = val_df
store['test_df'] = test_df
store.close()

../../feat4_trn_day8_val_day9


load

In [1]:
import pandas as pd

In [2]:
with pd.HDFStore('../input/feat/feat4_trn_day8_val_day9.h5') as store:
    train_df = store['train_df']
    val_df   = store['valid_df']
    test_df  = store['test_df']